# 네이버 플레이스 블로그 리뷰 크롤링
#### 2020-05-10 (일) 이한울
---
- 2020-05-10 (일): 플레이스 블로그 리뷰 긁어오기 및 스레딩 적용 완료
---
#### 2020-05-11 (월) 이한울
---
- 스레딩 실행 창에서 driver 경로를 본인의 크롬 드라이버가 있는 경로로 설정 후 실행
- time.sleep을 넉넉하게 주어야 HTTP 에러를 최대한 회피할 수 있습니다.
---
#### 2020-05-14 (목) 이한울
---
- 구글 드라이브 > 팀프로젝트 폴더 > [00]...마리톡 > [03] 데이터 > [03] 네이버 플레이스 리뷰 크롤링 > **naver_place_count_total_2020-05-01_final.csv** 파일을 다운로드, 원하는 경로로 이동시킨 뒤 진행해야 합니다.

In [17]:
import  urllib.request
import selenium
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from requests import ConnectionError, HTTPError, Timeout, TooManyRedirects
import urllib3

from pandas import Series, DataFrame
import pandas as pd
import numpy as np

from datetime import datetime
import time
import sys
import math
import re
from tqdm import tqdm, tqdm_notebook

import selenium.common.exceptions as sce
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.alert import Alert

from queue import Queue
from multiprocessing.pool import ThreadPool
from threading import Thread
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

import glob
import fnmatch

ModuleNotFoundError: No module named 'Exception'

In [2]:
# 플레이스의 업소별 ID를 담은 csv파일 DataFrame으로 변환
# 본인이 naver_place_count_total_2020-05-01_final.csv를 저장한 경로로 설정
df_place = \
DataFrame(pd.read_csv('./naver-place_csv/naver_place_count_total_2020-05-01_final.csv', sep=',', header=0, index_col=0))
df_place

,store_id,store_id_only_num,store_name,booking,receipt,blog,total
0,s1518997092,1518997092,준오헤어 노원사거리점,2015.0,3.0,15.0,2033.0
1,s1571970153,1571970153,차홍룸 홍대점,2009.0,1.0,10.0,2020.0
2,s1142939643,1142939643,차홍룸 용산센트럴점,1871.0,3.0,11.0,1885.0
3,s1481443590,1481443590,차홍룸 용산점,1541.0,2.0,11.0,1554.0
4,s1177346002,1177346002,차홍룸 양재점,1373.0,1.0,10.0,1384.0
...,...,...,...,...,...,...,...
2722,s38257322,38257322,바바헤어 대치점,1.0,1.0,2.0,4.0
2723,s1069634711,1069634711,정환헤어샵,1.0,1.0,2.0,4.0
2724,s20852762,20852762,머리짱헤어샵,1.0,2.0,1.0,4.0
2725,s36091089,36091089,살롱160,1.0,1.0,2.0,4.0


# 블로그 본문 크롤링 함수
## 위 블로그 리뷰 섬네일 크롤링 함수 모두 완료 후 진행해야 합니다!
#### 2020-05-13 (수) 이한울
---
- 헤드리스 적용
- 스레드 미구현
- **위 블로그 리뷰 섬네일 크롤링 함수 모두 완료 후 진행해야 합니다!**
---
#### 2020-05-14 (목) 이한울
---
- 스레드 구현 완료
- 본인이 저장한 CSV 파일의 경로와 이름명을 read_csv를 통해 DataFrame으로 변환한 뒤 실행하셔야 합니다.
- **반드시 위 리뷰 썸네일 크롤링 함수에서 본인의 범위까지 크롤링을 완료한 뒤 진행해야 합니다.**

In [3]:
# 블로그 본문을 저장할 빈 DataFrame 생성
df_text = DataFrame(columns=['store_id', 'store_name', 'link', 'main text', 'original_index'])

In [4]:
# 블로그리뷰 섬네일 저장 CSV 파일 읽어오기
# 경로 직접 설정하셔야 합니다.
csv_path = './place_review/2020-05-16_blog_no_text (0-362) (20004-25004).csv'
df_blog_review = pd.read_csv(csv_path, sep=',', header=0, index_col=0)
df_blog_review

,store_id,store_name,link,original_index
0,82379273,라트리헤어 스파힐 강남베르점,https://blog.naver.com/wldnjs2465/221745173453,174
1,82379273,라트리헤어 스파힐 강남베르점,https://blog.naver.com/wldnjs2465/221748402369,174
2,82379273,라트리헤어 스파힐 강남베르점,https://blog.naver.com/wldnjs2465/221740135665,174
3,82379273,라트리헤어 스파힐 강남베르점,https://blog.naver.com/wldnjs2465/221738311058,174
4,82379273,라트리헤어 스파힐 강남베르점,https://blog.naver.com/wldnjs2465/221806059676,174
...,...,...,...,...
4237,38310407,넌데일리,https://blog.naver.com/nondaily-jung/221033374741,299
4238,38310407,넌데일리,https://blog.naver.com/nondailyofficial/220811...,299
4239,38310407,넌데일리,https://blog.naver.com/nondaily-jung/221091345289,299
4240,38310407,넌데일리,https://blog.naver.com/nondailyofficial/220823...,299


In [5]:
df_no_text = DataFrame(columns=['store_id','store_name','link','original_index'])

In [6]:
# 에러가 발생한 블로그를 저장할 DataFrame 생성하기
df_text_error = DataFrame(columns=['store_id','store_name','link','original_index'])

In [7]:
def sub_emoji(content):
    
    find_txt = (content.text).strip().replace('\n', '')
    p = re.compile(r'[\\u\w]{1,}')
    filtered_txt = p.findall(find_txt)
    main_txt = ' '.join(filtered_txt)
    
    return main_txt

In [8]:
def main_text_save(main_txt, store_id, store_name, url):
    
    global df_text
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_text= df_text.append({
          'main text' : main_txt
        , 'store_id' : store_id
        , 'store_name' : store_name
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_text

In [9]:
# 에러 발생 시 따로 저장할 함수 선언
def text_error_save(store_id, store_name, url):
    
    global df_text_error
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_text_error = df_text_error.append({
          'store_id' : store_id
        , 'store_name' : store_name
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_text_error

In [10]:
def blog_no_text(store_id, store_name, url):
    
    global df_no_text
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_no_text = df_no_text.append({
          'store_id' : store_id
        , 'store_name' : store_name
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_no_text

In [11]:
def popup_detect(driver):
    try:
        alert = Alert(driver)
        print('팝업이 감지되었습니다. 확인 버튼을 누릅니다.')
        alert.accept()
        driver.close()
        driver.quit()
        return True
    
#     except sce.UnexpectedAlertPresentException:
#         alert = driver.switch_to_alert()
#         print('팝업 메시지:', alert.text)
#         alert.accept()
#         print('팝업이 감지되었습니다. 확인 버튼을 누릅니다.')
#         return True
    
    except sce.NoAlertPresentException:
        print('팝업이 감지되지 않았습니다. 크롤링을 진행합니다.')
        return False

In [12]:
def wait_xpath(driver, xpath):
    try:
        wait = WebDriverWait(driver, 10)
        elem = wait.until(EC.element_to_be_selected((By.XPATH, xpath)))

    except Exception:
        print('Element error..')
        print('browser refreshing')
        driver.refresh()
        time.sleep(2.5)
        return wait_xpath(driver, xpath)
    
    return elem

In [13]:
def wait_select(driver, selector):
    try:
        wait = WebDriverWait(driver, 10)
        elem = wait.until(EC.element_located_to_be_selected((By.CSS_SELECTOR, selector)))
    
    except Exception:
        print('Element error..')
        print('browser refreshing')
        driver.refresh()
        time.sleep(2.5)
        return wait_select(driver, selector)
    
    return elem

In [14]:
def driver_text_crawl(driver, url):

    global df_blog_review
    global df_text
    global df_place

    store_id = df_blog_review[df_blog_review['link'] == url]['store_id'].iloc[0]
    store_name = df_blog_review[df_blog_review['link'] == url]['store_name'].iloc[0]

    url = url.replace('m.', '')
    store = url
    
    driver.get(store)
    time.sleep(2)
    try:    
        if popup_detect(driver) == False:
            wait = WebDriverWait(driver, 10).until(lambda x: x.find_element(By.CSS_SELECTOR, '#mainFrame'))
            iframe = driver.find_element(By.CSS_SELECTOR, '#mainFrame')
            link = iframe.get_attribute('src')
            print(link)
            time.sleep(2)
            driver.get(link)
            driver.implicitly_wait(5)

            txt_path = '//div[@class="se_component_wrap sect_dsc __se_component_area"]'
            postview_path = '//div[@id="postViewArea"]'
            container_path = '//div[@class="se-main-container"]'

            for count in range(3): 
                try:
                    if count == 0:
                        blog_xpath = txt_path

                    elif count == 1:
                        blog_xpath = postview_path

                    else:
                        blog_xpath = container_path

                    wait = WebDriverWait(driver, 10).until(lambda x: x.find_element(By.XPATH, blog_xpath))
                    content = driver.find_element(By.XPATH, blog_xpath)
                    main_txt = sub_emoji(content)
                    main_text_save(main_txt, store_id, store_name, url)
                    print(main_txt)
                    break

                except sce.NoSuchElementException:    
                    print('Element is unable to find')

                    if count == 0:
                        print('txt_path 실패')

                    elif count == 1:
                        print('postview_path 실패')

                    else:
                        print('container_path 실패')
                        print('모두 실패하여 넘어갑니다.')
                        blog_no_text(store_id, store_name, url)

        else:
            blog_no_text(store_id, store_name, url) 
            
    except sce.TimeoutException:
        df_text.to_csv('./2020-05-19_place_blog-text_temp.csv')
        
    except Exception.
    
    print('END OF CRAWLING')
    time.sleep(np.random.rand() + np.random.rand())
    driver.close()
    driver.quit()        
    return df_text

# 업소별 블로그 리뷰 본문 텍스트 크롤링 스레딩
#### 2020-05-14 (목) 이한울
---
- 앞선 플레이스 블로그 리뷰 썸네일 크롤링 함수에서 저장된 **df_blog_review** DataFrame이나 **CSV파일**로 따로 저장해두었다면 해당 CSV파일을 DataFrame으로 읽은 후 해당 DataFrame의 길이만큼 범위를 지정해주시면 됩니다.
- a와 end 값을 본인의 범위에 맞게 지정해주신 뒤 실행하시면 됩니다.
- 간혹 지정된 범위까지 크롤링을 하였음에도 계속해서 실행되어 뒷 부분까지 크롤링하는 상황이 발생하니 유의해주시기 바랍니다.
- **함수가 다 실행된 후 CSV파일로 별도로 저장해야 합니다. (함수화 하지 않음)**

In [15]:
# 크롬 웹 드라이버 헤드리스 설정 및 웹 보안 무시
options = webdriver.ChromeOptions()

prefs = {
    'profile.default_content_setting_values': {
          'cookies': 2, 'images': 2, 'plugins': 2
        , 'popups': 2, 'geolocation': 2, 'notifications': 2
        , 'auto_select_certificate': 2, 'fullscreen': 2, 'mouselock': 2
        , 'mixed_script': 2, 'media_stream': 2, 'media_stream_mic': 2
        , 'media_stream_camera': 2, 'protocol_handlers': 2, 'ppapi_broker': 2
        , 'automatic_downloads': 2, 'midi_sysex': 2, 'push_messaging': 2
        , 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 'protected_media_identifier': 2
        , 'app_banner': 2, 'site_engagement': 2, 'durable_storage': 2
    }
}

options.add_experimental_option('prefs', prefs)
options.add_argument('--disable-dev-shm-usage')
options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36')
options.add_argument("Accept=text/html,application/xhtml+xml,application/xml;q=0.9,imgwebp,*/*;q=0.8")
options.add_argument("start-maximized")
options.add_argument('--no-sandbox')
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('headless')
options.add_argument("disable-gpu")
options.add_argument("--disable-web-security")

In [16]:
# 셀레니움 사용 플레이스 블로그 본문 크롤링 함수의 스레딩 적용

a = 1350 # 5759 #4583  # 4583 ~ 5564
start_no = 1350 # 5759
end = a + 5
end_df = 1850 # 5759

# csv 파일 저장을 위한 final csv파일의 start index를 저장
original_index_s = 0
original_index_e = 362
# =========================================================================

for i in tqdm_notebook(range(int(math.ceil((end_df - start_no) / 5)))):
    
    q = Queue(maxsize=5)
    for _ in range(q.maxsize):
        driver = Chrome('./chromedriver.exe', options=options)
        q.put(driver)
        
    print(str(a) + ' ~ ' + str(end - 1) + '의 블로그 본문 크롤링을 시작합니다.')
    with ThreadPool(5) as tp:
        # 스레드에 입력할 매개변수가 2개 이상일 경우 starmap을 사용한다.
        tp.starmap(
            driver_text_crawl,
            [(q.get(), df_blog_review['link'][ii]) for ii in range(a, end)]
        )
        
    a += 5
    end += 5
    time.sleep(0.5)
    # 100번 반복마다 본문을 수집할 때마다 DataFrame을 csv에 저장 (백업)
    if i in range(100, int(math.ceil(end_df / 5)), 100):
        df_text.to_csv('./2020-05-19_place_blog-text_TEMP (%i-%i) (%i-%i) %i.csv' %(original_index_s, original_index_e, start_no, end_df, i))
#         df_text_error.to_csv('./2020-05-19_place_blog-error_TEMP (%i-%i) (%i-%i).csv' %(original_index_s, original_index_e, start_no, end_df))
#         df_requests_error.to_csv('./2020-05-19_blog_requests-error_TEMP (%i-%i) (%i-%i).csv' %(original_index_s, original_index_e, start_no, end_df))
        df_no_text.to_csv('./2020-05-19_blog_no_text_TEMP (%i-%i) (%i-%i) %i.csv' %(original_index_s, original_index_e, start_no, end_df, i))
    
    if a > end_df:
        df_text.to_csv('./2020-05-19_place_blog-text_Real (%i-%i) (%i-%i) %i.csv' %(original_index_s, original_index_e, start_no, end_df, i))
#         df_text_error.to_csv('./2020-05-19_place_blog-error_Real (%i-%i) (%i-%i).csv' %(original_index_s, original_index_e, start_no, end_df))
#         df_requests_error.to_csv('./2020-05-19_blog_requests-error_Real (%i-%i) (%i-%i).csv' %(original_index_s, original_index_e, start_no, end_df))
        df_no_text.to_csv('./2020-05-19_blog_no_text_Real (%i-%i) (%i-%i) %i.csv' %(original_index_s, original_index_e, start_no, end_df, i))
        break

# ==========================================================================
# CSV파일 이중저장
# 정상적으로 본문 크롤링이 된 블로그 정보 CSV 파일로 저장
df_text.to_csv('./2020-05-19_place_blog-text_Real2 (%i-%i) (%i-%i).csv' %(original_index_s, original_index_e, start_no, end_df))

# 크롤링 도중 에러가 발생한 블로그 정보 CSV 파일로 저장
# df_text_error.to_csv('./2020-05-19_place_blog-error_Real2 (%i-%i) (%i-%i).csv' %(original_index_s, original_index_e, start_no, end_df))

# Requests Error가 발생한 블로그 정보 CSV 파일로 저장
# df_requests_error.to_csv('./2020-05-19_blog_requests-error_Real2 (%i-%i) (%i-%i).csv' %(original_index_s, original_index_e, start_no, end_df))

# 본문이 없거나, Attribute Error가 발생한 블로그 정보 CSV파일로 저장
df_no_text.to_csv('./2020-05-19_blog_no_text_Real2 (%i-%i) (%i-%i).csv' %(original_index_s, original_index_e, start_no, end_df))
# ==========================================================================

1350 ~ 1354의 블로그 본문 크롤링을 시작합니다.
팝업이 감지되었습니다. 확인 버튼을 누릅니다.
팝업이 감지되었습니다. 확인 버튼을 누릅니다.팝업이 감지되었습니다. 확인 버튼을 누릅니다.팝업이 감지되었습니다. 확인 버튼을 누릅니다.


팝업이 감지되지 않았습니다. 크롤링을 진행합니다.
팝업이 감지되지 않았습니다. 크롤링을 진행합니다.팝업이 감지되지 않았습니다. 크롤링을 진행합니다.팝업이 감지되지 않았습니다. 크롤링을 진행합니다.


https://blog.naver.com/PostView.nhn?blogId=zio7412&logNo=221164344773&redirect=Dlog&widgetTypeCall=true&directAccess=falsehttps://blog.naver.com/PostView.nhn?blogId=zio7412&logNo=220977382570&redirect=Dlog&widgetTypeCall=true&directAccess=falsehttps://blog.naver.com/PostView.nhn?blogId=zio7412&logNo=220937767699&redirect=Dlog&widgetTypeCall=true&directAccess=false


https://blog.naver.com/PostView.nhn?blogId=zio7412&logNo=221048907880&redirect=Dlog&widgetTypeCall=true&directAccess=false
팝업이 감지되었습니다. 확인 버튼을 누릅니다.
팝업이 감지되지 않았습니다. 크롤링을 진행합니다.
https://blog.naver.com/PostView.nhn?blogId=zio7412&logNo=220972030195&redirect=Dlog&widgetTypeCall=true&directAccess=false
END OF CRAWLING
END OF CRAWLING
END OF CRAWLING
END OF CRAWLING
END OF CRAWLING
135

C:\Users\82106\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: `item` has been deprecated and will be removed in a future version
  import sys


잠실역 미용실 박승철헤어스투디오 잠실홈플러스점 _ 헬시 시나몬 컬러 염색 잠실역미용실 잠실미용실 박승철헤어스투디오 잠실홈플러스점 2018 01 07 일요일 with 미오나 미오나의 신년 머리 스타일 변화를 위해 2018년 첫 주말에 찾아간 미용실 잠실역 미용실 박승철헤어스투디오 잠실홈플러스점박승철헤어스투디오 잠실홈플러스점서울특별시 송파구 올림픽로35가길 16 SONGPA FUN STADIUM예약상세보기서울 송파구 올림픽로35가길 16 지번 wnth 신천동 7 12 SONGPA FUN STADIUM 02 423 3325평일 10 00 22 00주말 10 00 20 302 4주 일요일 휴무잠실역 미용실 박승철 헤어스투디오 잠실홈플러스점은 지점 이름에서 바로 알 수 있듯이잠실 홈플러스에 있다 바깥이 보이는 투명 엘리베이터를 타고 4층으로 올라가면 된다 잠실홈플러스점 이기 때문에 마트 정기 휴무일인 2 4번째 일요일에 함께 휴무다 잘 알아두고 괜히 비장하게 머리 하러 갔다가 미용실은 커녕 마트 조차 입성을 하지 못하고 헛탕 치지 않도록 하자 주말의 미용실은 항상 아침부터 밤까지 붐비기 때문에 최대한 일찍 갔다 무려 오픈 시간인 10시에 냉큼 ㅋ가자마자 옷 벗고 짐 넣어 놓고 키 받음 앉아 있을 공간이 아늑하고 편안해서 좋은잠실역 미용실 박승철스투디오 잠실홈플러스점미용실에서는 장시간 머리 시술 받으면서 다소 답답함이 느껴질 수도 있는데 박승철 헤어스투디오 잠실홈플러스점은 한 쪽 외벽이 유리벽이라햇볕도 들어 오고 좀 탁 트인 기분이 들어 좋다 머리 하기 전의 미오나 머리 처음에는 아랫부분에 C컬 셋팅을 할까 했는데예전에 펌 했던 컬이 아직 제법 남아 있는 것 같고평소에 스타일링만 좀 더 신경 쓰면 컬은 좀 더 오래 살릴 수 있을 것 같아서염색을 하기로 염색과 함께 두피를 위한 두피팩 두피 트리트먼트 포함3가지를 하기로 함 오랜만에 하는 염색이라 신중하게 색깔을 고름 특히나 미오나의 근무지는 좀 보수적인 분위기라 많이 밝거나 튀는 머리색은 자제 해야 해서 단정하고 차분해 

누짱곱슬머리 오지고숱 겁나많은 녀자대체 나란 여자는왜 이리도 신경써야 할곳이 많은가지금 내 헤어 스타일이란 신승현샘 이 4개월전 만져준그 헤어스탈은 온데르 간데르 없다능걍 정준하단발됐다능4개월전에는분명 참한 스타일가볍고 찰랑한 가을녀자오른쪽은 과감히 현아머리 한얼굴이 나의 반쪽인 내 단짝쓰너의 예쁜얼굴을 가려미안하구나여튼 당시엔 좀 차분하니 이뻤다고 생각하나시간이 지나니ㅋㅋ이모지로도 드러나는나의 엄청난 숱과몬생김이 더해져서 그만 ㅜㅜ10년만에 머리를 기르는나로서는 미디길이 헤어스타일이 어색하기 짝이없어서하루에도 몇번씩 단발병 이 온다예전엔 좀 갠찮았던거 같은데 ㅋㅋ이라믄서 시종일관ㅋㅋ별로였을수도ㅋㅋ근데ㅋㅋ자기만 아는 괜 찮 은 그런거있자누 미안요 내 얼굴만 가립니다 ㅋㅋ 반포준오헤어 로 오게된 계기와 신승현디자이너 샘에게만 머리를 맡기는 사연은요 포스팅에 있으니 참고 반포 미용실 추천 준오헤어 신승현샘 곱슬 머리 잘하는 곳ㅎㅎ 안녕하세요 누짱입니다 D 얼마전 머리했어요 하핫 반포 미용실 추천 준오 헤어 신승현 샘 곱슬머리 m blog naver com광고아님 내돈 내산 후기각종 미담 포함 승현샘이 짱인 이유ㅋㅋㅋ거의 논문수준 체계적으로 쓴 폿흐팅이지ㄲ좋은 제품으로세심하게 케어승현샘은항상 좋은 제품으로 케어해주시기땜시내 개 돼지털도 아주 잘 길들여져서 난 ㅜㅜ 곱슬이 넘 심하고 머리털은 개털 순하디 순한 페르시안 고양이 털이 된것이다아 묻 따 로 맡기는 편이제는 아예 ㅋㅋ어떻게 머리할지 찾지도 않ㅋ미안요 ㅋㅋ샘ㅋㅋ항상 큰부담드림ㅋㅋJMT 먹방서비스오늘도 어김없이원하는 음료를 물어보시기에먹고싶은거 막 말해봄 바닐라 라떼 주세요ㅋㅋ 없는 메뉴 여서스텝분이 초코타서 카페모카 만들어주셨는데이거슨 한강 수준인것임ㅋㅋ적당히 안 달고 안 진해서 맛있었음승현샘에게 특별한거슨간식인디ㅋㅋ진짜 존맛탱ㅋㅋㅋㅋ뭐냐 달걀의 탱글함과마요네즈의 시큼함과 설탕의 달달함이입안에서 퐝퐝 진심 내가 좋아하는 그맛Instagram의 simsong_sim님 일요일 난 글씨도 그린다 좋아요 77개 댓글

얼룩과 손상이없는 탈색은준오헤어 시우부원장에게안녕하세요제법 괜찮은 장수미용사 시우부원장 입니다 날이 더워지면서 여름 휴가 벌써부터 생각들 하고 있습니다 여름철에는 야외도 많이 나가고 머리색도 밝게 탈색해서 예쁜 컬러 많이 하고 머리가가볍고 시원해 보이게 합니다 그래서 탈색할때는 손상도를 최소한으로줄이면서 색 얼룩 없이 깔끔하게 색이 빠져야 컬러를 예쁘게 입힐수 있습니다 탈색은 정말 정성과 꼼꼼하게 작업을 하지않으면 100 색 얼룩이 나오고 색이빠졌을때도 지저분하게 얼룩이 집니다 탈색을 많이 해보신 분들은 알고 있는사실이겠지만탈색 처음해보시는 분들은 일단모발이 밝게 됐다는 사실만으로 얼룩이 있는지 없는지 눈에 들어 오지 않습니다 다시 한번 강조 하지만 탈색은 얼룩이 없어야 합니다 고객님 탈색 시술전 사진 입니다 일단 버진 헤어이고 모발이 얇고 약간 밝은갈색 정도의 모발이지만 노랑색과 붉은색의멜라닌색소가 많이 있을것으로 예상이 됩니다 1차 탈색을 시술한 사진입니다 탈색을하면 두피쪽에 열이 많고 열기구나스티머를 사용하면 위쪽으로 더 밝게 나오고모발 아래쪽은 좀 더 어둡게 빠집니다 또는 고객님들의 과거 염색 시술의 역사에 따라서도탈색후의 결과물은 천차 만별입니다 그래서 탈색전에 충분히 상담을하고모발 진단을 잘 해야 합니다 염색을 이제 하지 않으려고블랙이나 진한 갈색으로 염색을 했는데 기억을 못하시는 고객님들 의외로 많아요 2차 탈색후 시술 사진입니다 모발 아래 붉은기가 많이 빠지고 전체적으로노랑기가 1차 탈색때 보다는 많이 없어졌네요1차 탈색후 높아진 모발 p h발란스를낮추는 작업을 해야 모발 손상도가 적습니다 대부분의 헤어 시술 펌 염색 탈색 은p h발란스를 원래 모발의 p h로 낮추지않으면컬러를 입히고나서 색상 유지가 오래가지 않고 물빠짐이 더 많으며 윤기도 없고 손상도가 높습니다 3차 탈색 시술 사진 입니다 좀 더 밝고 노랑기를 잡기 위해탈색약의 농도를 낮추고 꼼꼼하게 잘 바릅니다보통은 2차 탈색후 색을 입히는데보시는 3차 탈색정도 하시면 어떤색이든 다 입힐수가

...

요즘 날씨가 넘추워가지고아무곳도 나가기싫은거저만그런거아니죠 ㅠㅠ 레알 요즘 롱패딩없으면 밖돌아다니기가무서운거 인정 하시나요 여러분ㅎㅎㅎㅎㅎㅎㅎ약속때도 더지각하게되는이유가평소보다 이불에서 더나오기싫고그러다보니 늦게일어나서화장하랴 머리하랴ㅠㅠㅠㅠ복잡쓰 저는 원래 중국에서 거주중인데방학때마다 한국에와서머리를 하고가거든요자국민에게 머리를 맡겨야편한병에 걸려서그런지ㅎㅎ 방학때마다 꼭 미용실을 가는데이번에 엄마가 머리하신곳을추천해주셔서 다녀왔는데너무 만족해서후기포스팅을 안쓸수가 없었어요엄마가 추천해주신곳은영등포미용실인 엘라인헤어인데교통수단을 이용하신다면영등포역과 가까워요 영등포미용실엘라인서울특별시 영등포구 영중로3길 3예약상세보기영등포역 3번출구에서걸어서 4분거리니까굳이 차 가지고 안가도 될듯 하네용엘라인은 벌써부터크리스마스 준비에 한창이더라구요ㅋㅋㅋㅋㅋㅋㅋㅋ들어가자마자 귀여운 산타가 저랑 아이컨택 내부는 모노톤으로 깔끔했어요 지저분하지않고 청결상태도 좋았고엄마추천으로 왔다고하니까더 반겨주셔서 기분좋더라구요ㅎㅎㅎㅎㅎㅎㅎㅎ저는 엄마가 추천해주신별아 디자이너쌤께 시술받았어용디자이너쌤하고 나이대가 비슷해서신나가지고 떠드느라정작 머리는 안찍었다는 사실 포스팅해야겠다 생각하고 간게아니라서머리는 찍을생각도 못했는데머리가 너무 이쁘게나와서비포를 못찍은게 정말 한이에요ㅠ ㅠ요기도 크리스마스 ㅋㅋㅋㅋㅋㅋㅋ트리대신에 전구로 이렇게 장식해논것도참 예쁘고 독특하더라구요센스만점 요즘 좋은 미용실들은다 실내에 작은 카페가있는듯해요고객에 대한 CS를 기본으로 생각하는듯해서대우받는 느낌도들고좋은미용실에 왔구나 하는 느낌도 들어서좋더라구요ㅎㅎㅎ머리하는동안 심심하지않게잡지도 주시고커피랑 주전부리도 주셨어요ㅎㅎㅎㅎ내가좋아하는 커피과자ㅠㅠ커피랑먹으면 을매나 맛있게요 이상하게 집에서는 잡지 펼쳐보지도않는데미용실만오면 왜이렇게 잡지가 재미있는지혹시 저만 그런가요ㅎㅎㅎㅎ저는 아침에 머리를 말리고 출근해야하기때문에제일 시간이 단축되면서도쉬운손질할수있는 머리로 별아쌤하고 상담했어요쌤께서 C컬펌을 추천해주셨고머리가 

안녕하세용이번에 크리스마스도 다가오고곧 연말이기도 해서 머리 스타일변화좀 주고자해요 하고싶은 머리가 있었는데이번주말에는 꼭 해야지 하고 인터넷 폭풍 검색후주말에 다녀왔답니다 영등포타임스퀘어미용실엘라인헤어입니다 제가 하고싶었던 머리는레이어드c컬펌 이에요 히히영등포타임스퀘어미용실엘라인헤어 이곳에 왜 오게 되었냐면인터넷에 검색하고 여러후기를봤는데 엘라인헤어 미용실에수정디자이너선생님 께 레이어드컷 과파마를 받은 후기를 받는데 너무 이쁘게잘해주셨더라구요 그런 후기 글 들이 많아서찾아오게 되었어요 수정디자이너 선생님께레이어드c컬펌을 하고 싶다고조금 굵은c컬을 원한다고 말씀드렸더니샴푸후 모발상태 를 꼼꼼히 체크해주시더니모발상태에 맞는 펌제 와 가격을자세히 상담해주셨어요 그리고 모발끝이 조금 건조하기에크리닉을 같이 하는걸 추천해주셔서크리닉도 같이하기로 했답니다 모발이 건조하고 많이 상한상태이면컬이 이쁘게 안나오지않나요 연화를 하고 셋팅을 합니당 ㅎㅎ레이어드C컬펌 너무기대되요꺄아악 _ 셋팅을 해주시면서 셋팅은 제일 낮은온도도 100도 이상이라 극손상모로그나마 젤 낮은온도로 열처리를해주셨다고 했어요 블로그후기 를 보면서반신반의 했는데 수정디자이너분께서 정말로너무 친절하시고 꼼꼼하게해주시더라구요 엄지척 파마를 하면서기다리는동안 지루하지않게음료수랑 과자도챙겨주셔서 맛있게먹었답니다 머리 우선 레이어드컷한거 너무 맘에들어서C컬펌 이 얼마나 이쁘게 나올지기대너무 되요 막 샴푸를 하고 나온 모습이에용 어떤가요 _ 일단 머릿결이 굉장히좋아보이고웨이브가 이쁘게 나온거같죠 말린모습 보여드릴게요어모나 어머나 레이어드 컷도 너무 이쁘고펌이 자연스러우면서굉장히 여성스럽게 이뻐서러블리한 느낌이 나요진짜 제가 딱 원하던머리가 나와서 정말 대만족입니다제가 그 많은 영등포미용실 중이곳을 선택한건 신의한수였어요수정디자이너 선생님께서젖은상태에서 머리말려주시면서머리 말리는 방법 제품바르는방법 스타일링 하는방법까지 하나하나 차근차근설명해주셨어요 아래 컬이 너무이쁘지않나요C컬펌에 레이어드 컷을하니깐 심심해보이지도 

박준뷰티랩 구로디지털역점에서 봄바람 맞을 준비 완료벌써 약 9개월 가까이 엄마로만 살아온 나에게도 따스한 봄은 찾아오고 있었다임신 출산으로 2년 가까이 미용실 문앞에는 가보지도 못했는데 살랑살랑 불어오는 봄바람이 유혹하는 스타일변신 여자의 변신은 무죄라고 그리하여 2년만에 굽슬굽슬한 웨이브로 봄맞이 헤어변신을 하기로 했다주소 서울 구로구 구로동 디지털로 26길 지플러스타워 123번지 1층전화번호 02 6969 7979이마트 맞은편에 새로 오픈한 박준뷰티랩 구로디지털역점 오픈 이벤트로 3개월간 진행되는 1 1 이벤트 행사가격으로 펌 or 염색 시술 시 1명은 공짜로 시술할 수 있는 파격적인 이벤트가 진행중이다그래서인지 옆에서 봄맞이 단장하는 사람들은 전부다 커플이었다이벤트 기간동안 구로디지털단지 데이트 코스로 박준뷰티랩이 1위로 랭크될런지도 입구에 행사 가격과 회원가입 20 할인된 시술 금액이 안내되어 있다기장추가 없이 행사가에 시술할 수 있고 롯데 현대 갤러리아 신세계 백화점 상품권 사용도 가능하다오픈한지 얼마 안되어서 그런지 샤방샤방한 인테리어 거울 옆에는 무료한 시간을 달래줄 수 있는 태블릿 도 있는 최신식 시설의 구로디지털단지 미용실 미용실을 너무 안가서 그런건지 기다리는데도 설렌다 ㅋㅋㅋ상담을 위해 마련된 자리 한켠으로 마련된 도서와 헤어제품들 펌이나 염색 클리닉처럼 시간이 오래 걸리는 시술시에는 책도 읽을 수 있을 듯 하다자리에 앉아서 잠깐 둘러보는 사이에 담당 스타일리스트 분께서 오셔서 상담을 해주시는데 원하는 스타일 현재 모발상태 등을 간단히 체크하면서 상담을 해주신다VIP 정액권 구매시 시술시 30 할인을 받을 수 있는데 컷트도 할인이 되니 짧은 머리를 선호하는 사람이나 남성에게는 메리트 있는 혜택인 듯 하다초이 스타일리스트님과의 상담 후 어차피 머리 묶고 있는 시간이 더 많을테니 굵은 컬을 이용해서 말리기만 해도 손질한듯 안한듯 내츄럴하게 열펌을 하기로 했다펌을 하긴 하지만 컬이 많은 머리를 별로 안좋아하기도 하고 어차피 아가와 함께 지내다보면

가을분위기 물씬나는구디미용실 구로디지털단지 박준뷰티랩펌한지 거의 10달 다 되가는 동생 휴일을 맞아 가을분위기 물씬나는 헤어스타일링 해야겠다고 자매끼리 나들이 나섰답니다 머리를 자주하는 자매들이 아닌터라 한번할때 머리결 손상없이 이쁘게 잘해야 한답니다 그래서 솜씨좋은 미용실을 찾아가야하죠 그래서 다녀온 구디미용실 박준뷰티랩구로디지털단지점구디역에서 도보 5분 정도 서울디지털 국가산업단지내 G플러스타워 1층에 위치하고 있답니다 박준뷰티랩 구로디지털역점서울특별시 구로구 디지털로26길 123 지하 1층 013 014호예약상세보기박준뷰티랩이야 워낙 머리 잘하기로 유명한 곳이지만 구로디지털점은 또 그만의 독특한 분위기가 있네요 딱보기에 미용실인지 카페인지 구분안갈정도로 환하고 럭셔리한 인테리어까지 우리도 오전에 왔는데 휴일을 맞아 머리하러 오신분들이 참 많더라구요 헤어디자이너 선생님들 모두 엄청 바쁘심 잘나간다는 미용실은 요즘은 머리뿐만 아니라 a까지 있어야 하죠 핸드폰 충전은 기본이고 음료테이크아웃에 여성용품제공까지 한번 머리하면 몇시간을 있어야 하는 고객들을 위한 서비스가 다양하게 준비되어 있네요 주차 이용 서비스도 이렇게 진행되고 있대요 저희는 예약을 하고 왔지만 예약손님보다 먼저오신분들 부터 진행하는 우선대기시스템을 운영중이라고 해요 그래서 잠시 이렇게 상담석에 앉아 대기했답니다 중간에 카운터를 두고 뺑 둘러가면서 헤어스타일링 하는곳 대기하는곳 샴푸실 VIP실등이 있더라구요 안쪽에 이렇게 VIP실도 있던데 이런데서 머리 하시는분들은 어떤분인지 사뭇 궁금아마 개별적으로 관리받고 있는 분들인지 옆에는 이름이 빼곡히 적힌 상자들이 있네요 바람에 찰랑거리는 윤기 좔좔 흐르는 헤어스타일링 가지신분들 아마도 이렇게 관리 받고 계시겠죠 펌한지 거의 10달되서 부시시하게 다 풀려버리고 길기도 엄청 길어진 동생 머리에요 확실히 지저분해 보이는게 정리하긴 해야할듯 기장은 좀 잘라내고 가을에는 역시 S펌이죠 S펌으로 이쁘게 펌하기로 했답니다 우선 너무 길게 자란 머리 이쁘게 컷팅 들

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=51817): Max retries exceeded with url: /session/745fba46e9ecea6d22297da166b09d0c/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000024635D94088>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [21]:
df_no_text.to_csv('./2020-05-19_blog_no_text (0-362) (1350-1730).csv')

In [19]:
df_text.to_csv('./2020-05-19_place_blog_text (0-362) (1350-1730).csv')

In [ ]:
a = 10004 # 5759 #4583  # 4583 ~ 5564
start_no = 10004 # 5759
end_df = int(len(df_blog_review[start_no:15004]))
end_df

In [ ]:
df_blog_review[0:10]

In [ ]:
start_no = 0
end_df = 2032
original_index_s = 0
original_index_e = 362
df_text.to_csv('./place_review/2020-05-18_test (%i-%i) (%i-%i).csv' %(original_index_s, original_index_e, start_no, end_df))

In [ ]:
driver.switch_to.alert?

In [ ]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
sce.WebDriverException?

In [ ]:
sce.UnexpectedAlertPresentException?

In [ ]:
ConnectionRefusedError?

In [ ]:
driver.find_element?

In [ ]:
driver.implicitly_wait?

In [ ]:
EC.element_located_to_be_selected?

연습

In [ ]:
Alert?

In [ ]:
url = 'https://blog.naver.com/alldaybunny/221749223459'
driver = Chrome('./chromedriver.exe')

driver.get(url)
if popup_detect() == True:
    driver.close()
    driver.quit()
        
elif popup_detect() == False:
    driver.implicitly_wait(10)
    iframe = driver.find_element(By.CSS_SELECTOR, '#mainFrame')
    link = iframe.get_attribute('src')
    print(link)
    driver.implicitly_wait(10)
    driver.get(link)
    driver.implicitly_wait(5)

    txt_path = '//div[@class="se_component_wrap sect_dsc __se_component_area"]'
    postview_path = '//div[@id="postViewArea"]'
    container_path = '//div[@class="se-main-container"]'

    for count in range(3): 
        try:
            if count == 0:
                blog_xpath = txt_path

            elif count == 1:
                blog_xpath = postview_path

            else:
                blog_xpath = container_path

            driver.implicitly_wait(10)
            content = driver.find_element(By.XPATH, blog_xpath)
            main_txt = sub_emoji(content)
            main_text_save(main_txt, store_id, store_name, url)
            print(main_txt)
            break

        except sce.NoSuchElementException:    
            print('Element is unable to find')

            if count == 0:
                print('txt_path 실패')

            elif count == 1:
                print('postview_path 실패')

            else:
                print('container_path 실패')
                print('모두 실패하여 넘어갑니다.')

    print('END OF CRAWLING')
    time.sleep(1.5)
    driver.close()
    driver.quit()

In [ ]:
df_blog_review[0:10]

In [ ]:
EC.element_located_to_be_selected

In [3]:
WebDriverWait?